In [1]:
%matplotlib notebook
import os, sys
import logging
import random
import h5py
import shutil
import time
import argparse
import numpy as np
import sigpy.plot as pl
import torch
import sigpy as sp
import torchvision
from torch import optim
from tensorboardX import SummaryWriter
from torch.nn import functional as F
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib
# import custom libraries
from utils import transforms as T
from utils import subsample as ss
from utils import complex_utils as cplx
from utils.resnet2p1d import generate_model
from utils.flare_utils import roll
from utils import data_ut as dut
# import custom classes
from utils.datasets import SliceData
from subsample_fastmri import MaskFunc
from MoDL_single import UnrolledModel
import argparse
import matplotlib.pyplot as plt
%matplotlib inline
import nibabel as nib
from models.SAmodel import MyNetwork
from models.Unrolled import Unrolled
from models.UnrolledRef import UnrolledRef
from models.UnrolledTransformer import UnrolledTrans

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
%load_ext autoreload
%autoreload 0

In [2]:
!pwd

/home/tal/docker/MoDLsinglechannel/modl_singlechannel_reference


In [3]:
!which python3

/home/tal/docker/dockvenv/bin/python3


In [4]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [6]:
#checkpoint_file = "./L2_checkpoints_poisson_x2_SAunrolled/model_80.pt"
checkpoint_file = "./L2_checkpoints_poisson_x2_SAunrolledRef/model_80.pt"
#checkpoint_file = "./L2_checkpoints_poisson_x2_MoDL/model_80.pt"
#checkpoint_file = "./L2_checkpoints_poisson_x2_SAunrolledRef/model_20.pt"
checkpoint = torch.load(checkpoint_file,map_location=device)

In [7]:
params = checkpoint["params"]
#single_MoDL = UnrolledModel(params).to(device)
#single_MoDL = MyNetwork(2,2).to(device)
#single_MoDL = Unrolled(params).to(device)
single_MoDL = UnrolledRef(params).to(device)
#single_MoDL = UnrolledTrans(params).to(device)
single_MoDL.load_state_dict(checkpoint['model'])

shared weights


<All keys matched successfully>

In [8]:
class DataTransform:
    """
    Data Transformer for training unrolled reconstruction models.
    """

    def __init__(self, mask_func, args, use_seed=False):
        self.mask_func = mask_func
        self.use_seed = use_seed
        self.rng = np.random.RandomState()

    def __call__(self, kspace, target, reference_kspace,reference,slice):
        im_lowres = abs(sp.ifft(sp.resize(sp.resize(kspace,(256,24)),(256,160))))
        magnitude_vals = im_lowres.reshape(-1)
        k = int(round(0.05 * magnitude_vals.shape[0]))
        scale = magnitude_vals[magnitude_vals.argsort()[::-1][k]]
        kspace = kspace/scale
        target = target/scale
        # Convert everything from numpy arrays to tensors
        kspace_torch = cplx.to_tensor(kspace).float()   
        target_torch = cplx.to_tensor(target).float()   
        # Use poisson mask instead
        mask2 = sp.mri.poisson((256,160), 5, calib=(18, 14), dtype=float, crop_corner=False, return_density=True, seed=0, max_attempts=6, tol=0.01)
        mask2[128-10:128+9,80-8:80+7] = 1
        mask_torch = torch.stack([torch.tensor(mask2).float(),torch.tensor(mask2).float()],dim=2)
        mask_torch = T.kspace_crop(mask_torch,0.67)
        #kspace_torch = T.kspace_cut(mask_torch,0.5)
        kspace_torch = T.awgn_torch(kspace_torch,15,L=1)
        kspace_torch = kspace_torch*mask_torch

        ### Reference addition ###
        im_lowres_ref = abs(sp.ifft(sp.resize(sp.resize(reference_kspace,(256,24)),(256,160))))
        magnitude_vals_ref = im_lowres_ref.reshape(-1)
        k_ref = int(round(0.05 * magnitude_vals_ref.shape[0]))
        scale_ref = magnitude_vals_ref[magnitude_vals_ref.argsort()[::-1][k_ref]]
        reference = reference / scale_ref
        reference_torch = cplx.to_tensor(reference).float()

        return kspace_torch,target_torch,mask_torch, reference_torch

In [9]:
def create_datasets(args):
    # Generate k-t undersampling masks
    train_mask = MaskFunc([0.08],[4])
    train_data = SliceData(
        root=str(args.data_path),
        transform=DataTransform(train_mask, args),
        sample_rate=1
    )
    return train_data
def create_data_loaders(args):
    train_data = create_datasets(args)
#     print(train_data[0])

    train_loader = DataLoader(
        dataset=train_data,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=8,
        pin_memory=True,
    )
    return train_loader
def build_optim(args, params):
    optimizer = torch.optim.Adam(params, lr=args.lr, weight_decay=args.weight_decay)
    return optimizer

In [10]:
import numpy as np
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr, normalized_root_mse as nrmse
from skimage import img_as_float
from types import SimpleNamespace as Namespace


# Hyperparameters
params = Namespace()
params.data_path = "./test_data/patient29b"
params.batch_size = 1
params.num_grad_steps = 4
params.num_cg_steps = 8
params.share_weights = True
params.modl_lamda = 0.05
params.lr = 0.00001
params.weight_decay = 0
params.lr_step_size = 10
params.lr_gamma = 0.5
params.epoch = 21
params.reference_mode = 0
params.reference_lambda = 0.1

# Load test data
test_loader = create_data_loaders(params)

# Initialize lists to store metrics
mse_in_list, mse_out_list = [], []
psnr_in_list, psnr_out_list = [], []
ssim_in_list, ssim_out_list = [], []

single_MoDL.eval()  # Set model to evaluation mode

with torch.no_grad():  # Disable gradient computation for evaluation
    for data in test_loader:
        input, target, mask, reference = data
        input = input.to(device)
        reference = reference.to(device)
        mask = mask.to(device)
        # Forward pass through the model
        output = single_MoDL(input.float(),reference_image=reference, mask=mask)
        
        # Handle output
        kspace_out = T.fft2(output.cpu().squeeze(0))
        kspace_out = T.kspace_crop(kspace_out.squeeze(0),0.67)
        im_out = T.ifft2(kspace_out)
        
        kspace_target = T.fft2(target.cpu().squeeze(0))
        kspace_target = T.kspace_crop(kspace_target.squeeze(0),0.67)
        target = T.ifft2(kspace_target)

        
        #print(cplx.to_numpy(im_out.cpu()).shape)
        cplx_image_target = cplx.to_numpy(target.cpu())
        cplx_image_in = cplx.to_numpy(T.ifft2(input.cpu())).squeeze(0)
        cplx_image_out = cplx.to_numpy(im_out.cpu().squeeze(0))
        cplx_image_reference = cplx.to_numpy(reference.cpu()).squeeze(0)
    

        target_numpy = cplx.to_numpy(target.cpu())
        input_numpy = cplx.to_numpy(T.ifft2(input.cpu())).squeeze(0)
        out_numpy = cplx.to_numpy(output.cpu()).squeeze(0)

        target_numpy_norm = np.abs(target_numpy)/np.max(np.abs(target_numpy)).squeeze(0)
        input_numpy_norm = np.abs(input_numpy)/np.max(np.abs(input_numpy)).squeeze(0)
        out_numpy_norm = np.abs(out_numpy)/np.max(np.abs(out_numpy)).squeeze(0)

        """
        # Find comparison area:
        area = target_numpy_norm > 0.10
        kernel = np.ones((5, 5)) / 25.0
        area = convolve(area.squeeze(0), kernel, mode='constant', cval=0.0)
        area[area>0.1] = 1
        target_numpy_norm = target_numpy_norm * area
        input_numpy_norm = input_numpy_norm * area
        out_numpy_norm = out_numpy_norm * area
        """
        # Calculate metrics
        # Calculate SSIM values
        data_range = target_numpy_norm.max() - target_numpy_norm.min()

        ssim_in, _ = ssim(target_numpy_norm, input_numpy_norm, data_range=data_range, full=True)
        ssim_out, _ = ssim(target_numpy_norm, out_numpy_norm, data_range=data_range, full=True)


        # Calculate PSNR
        psnr_in = T.PSNR_numpy(target_numpy_norm, input_numpy_norm)
        psnr_out = T.PSNR_numpy(target_numpy_norm, out_numpy_norm)

        #print(img_in.shape)
        #print(img_out.shape)
        #print(img_target.shape)
        """
        plt_concat = np.concatenate((np.abs(cplx_image_reference),np.abs(cplx_image_in),np.abs(cplx_image_out),np.abs(cplx_image_target)),axis=1)
        fig, axs = plt.subplots(1, 1, figsize=(20, 5))  # 1 row, 3 columns
        im = axs.imshow(plt_concat, cmap='gray')
        axs.set_title(f'Reference                 Input (PSNR: {psnr_in:.2f})               Output (PSNR: {psnr_out:.2f})               Target')
        

        plt_concat = np.concatenate((np.abs( cplx.to_numpy(input.cpu()).squeeze(0)),np.abs(cplx.to_numpy(kspace_out)),np.abs(cplx.to_numpy(kspace_target))),axis=1)
        fig, axs = plt.subplots(1, 1, figsize=(20, 5))  # 1 row, 3 columns
        im = axs.imshow(np.log(plt_concat), cmap='gray')
        axs.set_title(f'Input (PSNR: {psnr_in:.2f})               Output (PSNR: {psnr_out:.2f})               Target')
        """   
        # Calculate MSE
        mse_in = np.mean(np.abs(input_numpy_norm-target_numpy_norm)**2)
        mse_out = np.mean(np.abs(out_numpy_norm-target_numpy_norm)**2)

        # Append metrics to lists
        mse_in_list.append(mse_in)
        mse_out_list.append(mse_out)
        psnr_in_list.append(psnr_in)
        psnr_out_list.append(psnr_out)
        ssim_in_list.append(ssim_in)
        ssim_out_list.append(ssim_out)


# Calculate and print average metrics
print(f'Average MSE input: {np.mean(mse_in_list)}')
print(f'Average MSE output: {np.mean(mse_out_list)}')
print(f'Average PSNR input: {np.mean(psnr_in_list)}')
print(f'Average PSNR output: {np.mean(psnr_out_list)}')
print(f'Average SSIM input: {np.mean(ssim_in_list):.4f}')
print(f'Average SSIM output: {np.mean(ssim_out_list):.4f}')

print(f'Test slices: {len(test_loader)}')

/home/tal/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 5, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Average MSE input: 0.004426472599056894
Average MSE output: 0.0014809640124440193
Average PSNR input: 23.604061656064967
Average PSNR output: 28.54786727302269
Average SSIM input: 0.3855
Average SSIM output: 0.5526
Test slices: 56
